Pull total number of orders that were completed on 18th March 2023

Pull total number of orders that were completed on 18th March 2023 with the first name ‘John’ and last name Doe’

Pull total number of customers that purchased in January 2023 and the average amount spend per customer

Pull the departments that generated less than $600 in 2022

What is the most and least revenue we have generated by an order

What were the orders that were purchased in our most lucrative order

TABLE INFO :

SALES – Date, Order_id, Item_id, Customer_id, Quantity, Revenue

ITEMS – Item_id, Item_name, price, department

CUSTOMERS- customer_id, first_name,last_name,Address

Small Dataset can be generated using chatgpt.

In [29]:
import sqlite3

# Connect to an in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables
cursor.execute('''
    CREATE TABLE SALES (
        Date TEXT, Order_id INTEGER, Item_id INTEGER, Customer_id INTEGER, Quantity INTEGER, Revenue REAL
    )
''')
cursor.execute('''
    CREATE TABLE ITEMS (
        Item_id INTEGER, Item_name TEXT, price REAL, department TEXT
    )
''')
cursor.execute('''
    CREATE TABLE CUSTOMERS (
        Customer_id INTEGER, first_name TEXT, last_name TEXT, Address TEXT
    )
''')

# Insert data
sales_data = [
    ('2023-03-18', 1, 101, 1, 2, 50.0),   # John Doe, March 18
    ('2023-03-18', 2, 102, 2, 1, 30.0),   # Jane Smith, March 18
    ('2023-01-15', 3, 103, 1, 3, 75.0),   # John Doe, January
    ('2023-01-20', 4, 104, 3, 1, 100.0),  # Bob Jones, January
    ('2022-05-10', 5, 105, 2, 1, 500.0),  # Jane Smith, 2022
    ('2022-07-15', 6, 106, 3, 2, 20.0),   # Bob Jones, 2022
]
items_data = [
    (101, 'Shirt', 25.0, 'Clothing'),
    (102, 'Pants', 30.0, 'Clothing'),
    (103, 'Hat', 25.0, 'Accessories'),
    (104, 'Jacket', 100.0, 'Clothing'),
    (105, 'Coat', 500.0, 'Clothing'),
    (106, 'Socks', 10.0, 'Accessories'),
]
customers_data = [
    (1, 'John', 'Doe', '123 Main St'),
    (2, 'Jane', 'Smith', '456 Oak Ave'),
    (3, 'Bob', 'Jones', '789 Pine Rd'),
]

cursor.executemany('INSERT INTO SALES VALUES (?, ?, ?, ?, ?, ?)', sales_data)
cursor.executemany('INSERT INTO ITEMS VALUES (?, ?, ?, ?)', items_data)
cursor.executemany('INSERT INTO CUSTOMERS VALUES (?, ?, ?, ?)', customers_data)
conn.commit()

In [31]:
cursor.execute("SELECT COUNT(DISTINCT Order_id) AS total_orders FROM SALES WHERE Date = '2023-03-18'")
result = cursor.fetchone()
print(f"Total orders on March 18, 2023: {result[0]}")

Total orders on March 18, 2023: 2


In [33]:
cursor.execute("""
    SELECT COUNT(DISTINCT s.Order_id) AS john_doe_orders 
    FROM SALES s 
    JOIN CUSTOMERS c ON s.Customer_id = c.Customer_id 
    WHERE s.Date = '2023-03-18' 
    AND c.first_name = 'John' 
    AND c.last_name = 'Doe'
""")
result = cursor.fetchone()
print(f"John Doe's orders on March 18, 2023: {result[0]}")

John Doe's orders on March 18, 2023: 1


In [35]:
cursor.execute("""
    SELECT COUNT(DISTINCT t.Customer_id) AS total_customers, 
           AVG(t.total_spend) AS avg_spend_per_customer 
    FROM (
        SELECT Customer_id, SUM(Revenue) AS total_spend 
        FROM SALES 
        WHERE Date BETWEEN '2023-01-01' AND '2023-01-31' 
        GROUP BY Customer_id
    ) t
""")
result = cursor.fetchone()
print(f"Customers in January 2023: {result[0]}, Average spend per customer: {result[1]}")

Customers in January 2023: 2, Average spend per customer: 87.5


In [37]:
cursor.execute("""
    SELECT i.department, SUM(s.Revenue) AS total_revenue 
    FROM SALES s 
    JOIN ITEMS i ON s.Item_id = i.Item_id 
    WHERE s.Date BETWEEN '2022-01-01' AND '2022-12-31' 
    GROUP BY i.department 
    HAVING SUM(s.Revenue) < 600
""")
results = cursor.fetchall()
print("Departments with less than $600 in 2022:")
for dept, revenue in results:
    print(f"  {dept}: ${revenue}")

Departments with less than $600 in 2022:
  Accessories: $20.0
  Clothing: $500.0


In [39]:
cursor.execute("""
    SELECT MAX(order_revenue) AS max_revenue, 
           MIN(order_revenue) AS min_revenue 
    FROM (
        SELECT Order_id, SUM(Revenue) AS order_revenue 
        FROM SALES 
        GROUP BY Order_id
    ) t
""")
result = cursor.fetchone()
print(f"Most revenue by an order: ${result[0]}, Least revenue by an order: ${result[1]}")

Most revenue by an order: $500.0, Least revenue by an order: $20.0


In [41]:
cursor.execute("""
    WITH OrderRevenues AS (
        SELECT Order_id, SUM(Revenue) AS total_revenue 
        FROM SALES 
        GROUP BY Order_id
    )
    SELECT s.Order_id, s.Date, s.Item_id, i.Item_name, s.Quantity, s.Revenue 
    FROM SALES s 
    JOIN ITEMS i ON s.Item_id = i.Item_id 
    WHERE s.Order_id = (
        SELECT Order_id 
        FROM OrderRevenues 
        ORDER BY total_revenue DESC 
        LIMIT 1
    )
""")
results = cursor.fetchall()
print("Details of the most lucrative order:")
for row in results:
    print(f"  Order {row[0]}, Date: {row[1]}, Item: {row[3]}, Quantity: {row[4]}, Revenue: ${row[5]}")

Details of the most lucrative order:
  Order 5, Date: 2022-05-10, Item: Coat, Quantity: 1, Revenue: $500.0
